Train Model

In [ ]:
prefix = "DEMO-scikit-decision-tree-iris"
import boto3
import re
import os
import numpy as np
import pandas as pd
import sagemaker
from sagemaker import get_execution_role
from time import gmtime, strftime
from sagemaker.serializers import CSVSerializer

In [ ]:
role = get_execution_role()
sess = sagemaker.Session()
print('Role: ',role)

In [ ]:
WORK_DIRECTORY = "data"
data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [ ]:
account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name
image = "{}.dkr.ecr.{}.amazonaws.com/sagemaker-decision-trees:latest".format(account, region)

tree = sagemaker.estimator.Estimator(
    image,
    role,
    1,
    "ml.c4.2xlarge",
    output_path="s3://{}/output".format(sess.default_bucket()),
    sagemaker_session=sess,
)

tree.fit(data_location)

Host Model

In [ ]:
deployer = tree.deploy(1, "ml.m4.xlarge", serializer=csv_serializer)

In [ ]:
predictor = Predictor(endpoint_name=deployer.endpoint, sagemaker_session=sess,serializer=CSVSerializer())
for i in range(0,10):
    print(predictor.predict([1,2,3,4]).decode("utf-8").replace('\n',''))